**Import datasetu**
https://www.kaggle.com/datasets/tunguz/covid19-finance-sector-related-policy-responses


In [101]:
import pandas as pd

file_path = 'sources\\covid-fci-data.csv'

df = pd.read_csv(file_path, encoding='mac_roman')
df.head()

,ID,Country Name,Country ISO3,Income Level,Authority,Date,Level 1 policy measures,Level 2 policy measures,Level 3 policy measures,Details of the measure,Reference,Termination Date,Modification of Parent Measure,Parent Measure
0,1,China,CHN,Upper middle income,SUP,2/1/20,Banking sector,Operational continuity ...,Blank ...,Require banks and insurance companies to ensur...,http://www.gov.cn/zhengce/zhengceku/2020-02/03...,NaN,No,NaN
1,2,Canada,CAN,High income,CB,3/13/20,Liquidity/funding,Liquidity (incl FX)/ELA ...,Blank ...,BoC also created a Bankers Acceptance Purchase...,https://www.bankofcanada.ca/markets/market-ope...,10/31/20,No,NaN
2,3,China,CHN,Upper middle income,CB,2/5/20,Banking sector,Integrity ...,Other integrity ...,The PBC has issued Notices on AML/CFT requirem...,...,NaN,No,NaN
3,4,Thailand,THA,Upper middle income,CB,2/5/20,Liquidity/funding,Policy rate ...,Blank ...,BOT cut its benchmark interest rate by 25 basi...,https://www.bot.or.th/English/PressandSpeeches...,NaN,No,NaN
4,5,Russian Federation,RUS,Upper middle income,CB,2/7/20,Liquidity/funding,Policy rate ...,Blank ...,Policy rates were reduced by 25 basis points t...,https://www.cnbc.com/2020/02/07/reuters-americ...,NaN,No,NaN


**Sprawdzenie jak dużo NaN**

In [102]:
print("Ammount of all samples: ", len(df))

missing_data = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_percent': (df.isna().sum() / len(df)) * 100,
    'unique_values': df.nunique()
})

missing_data = missing_data.sort_values('missing_count', ascending=False)
missing_data

Ammount of all samples:  3723


,missing_count,missing_percent,unique_values
Parent Measure,3175,85.280688,242
Termination Date,2983,80.123556,98
Date,1,0.026860,262
Country ISO3,0,0.000000,157
Country Name,0,0.000000,157
ID,0,0.000000,2872
Authority,0,0.000000,7
Income Level,0,0.000000,6
Level 1 policy measures,0,0.000000,5
Level 2 policy measures,0,0.000000,19


**Podstawowa analiza**

In [103]:
df.dtypes

ID                                  int64
Country Name                       object
Country ISO3                       object
Income Level                       object
Authority                          object
Date                               object
Level 1 policy measures            object
Level 2 policy measures            object
Level 3 policy measures            object
Details of the measure             object
Reference                          object
Termination Date                   object
Modification of Parent Measure     object
Parent Measure                    float64
dtype: object

Zamiana dat [object >> date]

In [104]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y', errors='coerce')
df['Termination Date'] = pd.to_datetime(df['Termination Date'], format='%m/%d/%y', errors='coerce')
df[['Date', 'Termination Date']].head(10)


,Date,Termination Date
0,2020-02-01,NaT
1,2020-03-13,2020-10-31
2,2020-02-05,NaT
3,2020-02-05,NaT
4,2020-02-07,NaT
5,2020-02-19,NaT
6,2020-02-19,NaT
7,2020-02-17,NaT
8,2020-02-20,NaT
9,2020-02-20,NaT


Standaryzacja etykiet - czy aby napewno są spójne (brak literówek, różnic w wielkości liter itd.)

In [105]:
# Lista kolumn do oczyszczenia
columns_to_clean = [
    'Country Name',
    'Income Level',
    'Authority',
    'Level 1 policy measures',
    'Level 2 policy measures',
    'Level 3 policy measures',
    'Modification of Parent Measure'
]

for col in columns_to_clean:
    df[col] = (
        df[col]
        .astype(str)               # zamienia NaN na 'nan'
        .str.strip()               # usuwa spacje z początku i końca
        .str.lower()               # wszystko małymi literami
        .str.capitalize()          # pierwsza litera wielka, reszta mała
        .replace('Nan', pd.NA)     # zamienia string 'nan' z powrotem na NaN
    )
    
"""for col in columns_to_clean:
    print(f"\n{col} values:")
    print(df[col].value_counts(dropna=False))"""

'for col in columns_to_clean:\n    print(f"\n{col} values:")\n    print(df[col].value_counts(dropna=False))'

Czyszczenie pól typu „Blank” - kolumna Level 3 policy measures --> zamiana na NA

In [106]:
# Lista podejrzanych stringów do potraktowania jako brak danych
suspect_values = ['blank', 'none', 'n/a', 'na', 'unknown']

# Skan wszystkich kolumn
for col in df.columns:
    suspect_count = df[col].astype(str).str.strip().str.lower().isin(suspect_values).sum()
    if suspect_count > 0:
        print(f"{col}: {suspect_count} suspect values")

df['Level 3 policy measures'] = df['Level 3 policy measures'].replace(
    to_replace=['Blank', 'blank', 'BLANK'], value=pd.NA
)

Level 3 policy measures: 1292 suspect values


Sprawdzenie czy id jest unikalne

In [107]:
df['ID'].is_unique  # Powinno zwrócić: True

False

In [108]:
dupes = df[df['ID'].duplicated(keep=False)]
dupes.sort_values('ID')

,ID,Country Name,Country ISO3,Income Level,Authority,Date,Level 1 policy measures,Level 2 policy measures,Level 3 policy measures,Details of the measure,Reference,Termination Date,Modification of Parent Measure,Parent Measure
2816,37,France,FRA,High income,Sup,2020-03-03,Banking sector,Operational continuity,<NA>,SSM ensures that banks take necessary steps to...,https://www.bankingsupervision.europa.eu/press...,NaT,No,NaN
2818,37,Italy,ITA,High income,Sup,2020-03-03,Banking sector,Operational continuity,<NA>,SSM ensures that banks take necessary steps to...,https://www.bankingsupervision.europa.eu/press...,NaT,No,NaN
2819,37,Latvia,LVA,High income,Sup,2020-03-03,Banking sector,Operational continuity,<NA>,SSM ensures that banks take necessary steps to...,https://www.bankingsupervision.europa.eu/press...,NaT,No,NaN
2821,37,Slovak republic,SVK,High income,Sup,2020-03-03,Banking sector,Operational continuity,<NA>,SSM ensures that banks take necessary steps to...,https://www.bankingsupervision.europa.eu/press...,NaT,No,NaN
2815,37,Estonia,EST,High income,Sup,2020-03-03,Banking sector,Operational continuity,<NA>,SSM ensures that banks take necessary steps to...,https://www.bankingsupervision.europa.eu/press...,NaT,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,3861,Hungary,HUN,High income,Other,2020-12-02,Banking sector,Prudential,Supervisory guidance and expectations,The EBA communicated on 21 September the phase...,https://www.eba.europa.eu/regulation-and-polic...,2021-03-31,Other,1228.0
3583,3861,Estonia,EST,High income,Other,2020-12-02,Banking sector,Prudential,Supervisory guidance and expectations,The EBA communicated on 21 September the phase...,https://www.eba.europa.eu/regulation-and-polic...,2021-03-31,Other,1228.0
3591,3861,Romania,ROU,Upper middle income,Other,2020-12-02,Banking sector,Prudential,Supervisory guidance and expectations,The EBA communicated on 21 September the phase...,https://www.eba.europa.eu/regulation-and-polic...,2021-03-31,Other,1228.0
3593,3861,Slovenia,SVN,High income,Other,2020-12-02,Banking sector,Prudential,Supervisory guidance and expectations,The EBA communicated on 21 September the phase...,https://www.eba.europa.eu/regulation-and-polic...,2021-03-31,Other,1228.0


Wniosek - Id nie jest unikalne:

Dokładnie — to nie są duplikaty sensu stricto, tylko rekordy współdzielące to samo ID, bo odnoszą się do tej samej polityki zastosowanej w różnych krajach. Typowy przykład:
👉 Jedna decyzja nadzorcy europejskiego (SSM, EBA itd.) → wiele krajów objętych jednym działaniem.

Tworzenie dodatkowej kolumny Record_id

In [109]:
# Zamiana kolumny 'ID' na 'Original_ID' – zachowujemy oryginalny identyfikator z pliku
df.rename(columns={'ID': 'Original_ID'}, inplace=True)

# Tworzenie nowego unikalnego ID jako indeksu
df.reset_index(drop=True, inplace=True)
df['ID'] = df.index + 1
df.set_index('ID', inplace=True)

df.head()

,Original_ID,Country Name,Country ISO3,Income Level,Authority,Date,Level 1 policy measures,Level 2 policy measures,Level 3 policy measures,Details of the measure,Reference,Termination Date,Modification of Parent Measure,Parent Measure
ID,,,,,,,,,,,,,,
1,1,China,CHN,Upper middle income,Sup,2020-02-01,Banking sector,Operational continuity,<NA>,Require banks and insurance companies to ensur...,http://www.gov.cn/zhengce/zhengceku/2020-02/03...,NaT,No,NaN
2,2,Canada,CAN,High income,Cb,2020-03-13,Liquidity/funding,Liquidity (incl fx)/ela,<NA>,BoC also created a Bankers Acceptance Purchase...,https://www.bankofcanada.ca/markets/market-ope...,2020-10-31,No,NaN
3,3,China,CHN,Upper middle income,Cb,2020-02-05,Banking sector,Integrity,Other integrity,The PBC has issued Notices on AML/CFT requirem...,...,NaT,No,NaN
4,4,Thailand,THA,Upper middle income,Cb,2020-02-05,Liquidity/funding,Policy rate,<NA>,BOT cut its benchmark interest rate by 25 basi...,https://www.bot.or.th/English/PressandSpeeches...,NaT,No,NaN
5,5,Russian federation,RUS,Upper middle income,Cb,2020-02-07,Liquidity/funding,Policy rate,<NA>,Policy rates were reduced by 25 basis points t...,https://www.cnbc.com/2020/02/07/reuters-americ...,NaT,No,NaN


Usunięcie nadmiarowych znaków białych z kolumn tekstowych

In [110]:
# Usunięcie nadmiarowych znaków białych z końca i początku w tekstowych kolumnach
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

Usunięcie znaków nowej linii

In [111]:
# Usunięcie znaków nowej linii i zbędnych białych znaków z pól tekstowych
import re

for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(
        lambda x: re.sub(r'[\r\n]+', ' ', x).strip() if isinstance(x, str) else x
    )

Eksport poprawionych danych do .csv

In [112]:
# Eksport finalnych danych do pliku CSV
output_path = 'sources/covid-fci-data-cleaned.csv'

# Eksport z indeksem (ID jako index i jednocześnie PRIMARY KEY)
df.to_csv(output_path, index=True)

print(f"Zapisano do pliku: {output_path}")

df.head()


Zapisano do pliku: sources/covid-fci-data-cleaned.csv


,Original_ID,Country Name,Country ISO3,Income Level,Authority,Date,Level 1 policy measures,Level 2 policy measures,Level 3 policy measures,Details of the measure,Reference,Termination Date,Modification of Parent Measure,Parent Measure
ID,,,,,,,,,,,,,,
1,1,China,CHN,Upper middle income,Sup,2020-02-01,Banking sector,Operational continuity,<NA>,Require banks and insurance companies to ensur...,http://www.gov.cn/zhengce/zhengceku/2020-02/03...,NaT,No,NaN
2,2,Canada,CAN,High income,Cb,2020-03-13,Liquidity/funding,Liquidity (incl fx)/ela,<NA>,BoC also created a Bankers Acceptance Purchase...,https://www.bankofcanada.ca/markets/market-ope...,2020-10-31,No,NaN
3,3,China,CHN,Upper middle income,Cb,2020-02-05,Banking sector,Integrity,Other integrity,The PBC has issued Notices on AML/CFT requirem...,,NaT,No,NaN
4,4,Thailand,THA,Upper middle income,Cb,2020-02-05,Liquidity/funding,Policy rate,<NA>,BOT cut its benchmark interest rate by 25 basi...,https://www.bot.or.th/English/PressandSpeeches...,NaT,No,NaN
5,5,Russian federation,RUS,Upper middle income,Cb,2020-02-07,Liquidity/funding,Policy rate,<NA>,Policy rates were reduced by 25 basis points t...,https://www.cnbc.com/2020/02/07/reuters-americ...,NaT,No,NaN
